In [ ]:
!pip install PyGithub requests

In [ ]:
import pandas as pd
from github import Github
from tqdm import tqdm
import re

# please put the bug reports that you want to retrieve into a .csv file
file_name = ''
df = pd.read_csv('/content/drive/MyDrive/data_only/PBR/file_name.csv')

# please put your github OAuth code
OAuth = ''

g = Github(OAuth)
repo_list = ['tensorflow/tensorflow','keras-team/keras','pytorch/pytorch','apache/incubator-mxnet','BVLC/caffe']
for repository in repo_list:
  df2 = pd.DataFrame(columns=['Repository','Number','State','Title','Body','Labels','Comments','Codes','Commands','class','related'])
  repo = g.get_repo(repository)
  repository = repository.split('/')[1]
  print(f'\n Running {repository}')
  nums = df[df['Repository']==repository][['Number','class','related']]
  for index in tqdm(range(len(nums))):
    num = int(nums['Number'].iloc[index])
    issue = repo.get_issue(number=num)

    label_list = []
    for label in issue.labels:
      label_list.append(label.name)

    comment_list = []
    for comment in issue.get_comments():
      comment_list.append(comment.body)

    pattern = re.compile(r'```([^`]*)```')
    code_list = re.findall(pattern,issue.body)

    pattern = re.compile(r'`([^`]*)`')
    command_list = re.findall(pattern,re.sub(r'```([^`]*)```','',issue.body))
    
    text = issue.body
    if pd.isna(text):
      text = ''
    # text = re.sub(r'```([^`]*)```','',text)
    text = re.sub(r'`([^`]*)`','',text)

    df2 = df2.append({'Repository':repository,'Number':num,'State':issue.state,'Title':issue.title,'Body':text,'Labels':' '.join(label_list),'Comments':str(comment_list),'Codes':str(code_list),'Commands':str(command_list),'class':nums['class'].iloc[index],'related':nums['related'].iloc[index]} ,ignore_index=True)
  df2.to_csv(f'{repository}.csv')
